In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import os
print(os.getcwd())

e:\Bank Loan\drive-download-20251005T104132Z-1-001


In [8]:
Data = pd.read_csv('E:/Bank Loan/drive-download-20251005T104132Z-1-001/financial_loan.csv.csv')

In [9]:
Data.head(5)

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,...,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,11-02-2021,13-09-2021,13-04-2021,...,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,01-01-2021,14-12-2021,15-01-2021,...,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,05-01-2021,12-12-2021,09-01-2021,...,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,25-02-2021,12-12-2021,12-03-2021,...,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,01-01-2021,14-12-2021,15-01-2021,...,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835


In [11]:
df = Data.copy()

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38576 entries, 0 to 38575
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     38576 non-null  int64  
 1   address_state          38576 non-null  object 
 2   application_type       38576 non-null  object 
 3   emp_length             38576 non-null  object 
 4   emp_title              37138 non-null  object 
 5   grade                  38576 non-null  object 
 6   home_ownership         38576 non-null  object 
 7   issue_date             38576 non-null  object 
 8   last_credit_pull_date  38576 non-null  object 
 9   last_payment_date      38576 non-null  object 
 10  loan_status            38576 non-null  object 
 11  next_payment_date      38576 non-null  object 
 12  member_id              38576 non-null  int64  
 13  purpose                38576 non-null  object 
 14  sub_grade              38576 non-null  object 
 15  te

In [13]:
df.shape

(38576, 24)

In [15]:
clean_df = df.copy()

In [97]:
# 1): Remove duplicate rows
clean_df.drop_duplicates(inplace=True)
clean_df.shape

(38576, 24)

In [98]:
# 2): Convert all date columns to 'date' type (YYYY-MM-DD)

date_columns = ["issue_date", "last_credit_pull_date", "last_payment_date", "next_payment_date"]

for col in date_columns:
    # Convert to datetime first, specifying dayfirst=True
    df[col] = pd.to_datetime(df[col], format="%d-%m-%Y", errors='coerce', dayfirst=True)
    
    # Then extract only the date part (removes time)
    df[col] = df[col].dt.date

In [99]:
clean_df[date_columns].head(5)

,issue_date,last_credit_pull_date,last_payment_date,next_payment_date
0,2021-02-11,2021-09-13,2021-04-13,2021-05-13
1,2021-01-01,2021-12-14,2021-01-15,2021-02-15
2,2021-01-05,2021-12-12,2021-01-09,2021-02-09
3,2021-02-25,2021-12-12,2021-03-12,2021-04-12
4,2021-01-01,2021-12-14,2021-01-15,2021-02-15


In [100]:
# 3): Clean 'emp_length' column

# Step 1 — raw unique values
print(df["emp_length"].unique())

# Step 2 — after replacement
step1 = df["emp_length"].astype(str).replace(
    {"10+ years": "10", "< 1 year": "0.5", "<1 year": "0.5", "n/a": None})
print(step1.unique())

# Step 3 — after extraction
step2 = step1.str.extract(r'(\d+\.?\d*)')[0]
print(step2.unique())

# Step 4 — after numeric conversion
clean_df["emp_length"] = step2.astype(float)
print(clean_df["emp_length"].value_counts(dropna=False))

['< 1 year' '9 years' '4 years' '10+ years' '3 years' '5 years' '1 year'
 '6 years' '2 years' '7 years' '8 years']
['0.5' '9 years' '4 years' '10' '3 years' '5 years' '1 year' '6 years'
 '2 years' '7 years' '8 years']
['0.5' '9' '4' '10' '3' '5' '1' '6' '2' '7' '8']
emp_length
10.0    8870
0.5     4575
2.0     4382
3.0     4088
4.0     3428
5.0     3273
1.0     3229
6.0     2228
7.0     1772
8.0     1476
9.0     1255
Name: count, dtype: int64


In [101]:
clean_df["emp_length"]

0         0.5
1         9.0
2         4.0
3         0.5
4        10.0
         ... 
38571     0.5
38572     8.0
38573     5.0
38574     5.0
38575     4.0
Name: emp_length, Length: 38576, dtype: float64

In [102]:
# 4): Handle missing 'emp_title' values
clean_df["emp_title"] = clean_df["emp_title"].fillna("Not Provided")

In [103]:
clean_df["emp_title"].isna().sum()

np.int64(0)

In [104]:
# 5): Clean 'term' column (remove 'months' and make numeric)
clean_df["term"] = (
    clean_df["term"].astype(str)
    .str.replace(" months", "", regex=False)
    .str.replace(" month", "", regex=False)
)

In [105]:
clean_df["term"].head(5)

0    60
1    36
2    36
3    60
4    36
Name: term, dtype: object

In [109]:
print("\n Cleaned Data Overview:\n")
print(clean_df.dtypes)


 Cleaned Data Overview:

id                         int64
address_state             object
application_type          object
emp_length               float64
emp_title                 object
grade                     object
home_ownership            object
issue_date                object
last_credit_pull_date     object
last_payment_date         object
loan_status               object
next_payment_date         object
member_id                  int64
purpose                   object
sub_grade                 object
term                      object
verification_status       object
annual_income            float64
dti                      float64
installment              float64
int_rate                 float64
loan_amount                int64
total_acc                  int64
total_payment              int64
dtype: object


In [ ]:
# Downloading the new/edited Datasheet
clean_df.to_csv("Clean_Loan_Data.csv.csv", index=False)